In [8]:
# 必要なPythonライブラリをインポート
import boto3
import json
import urllib.request
from dotenv import load_dotenv
from rich import print

# .envファイルから環境変数を読み込む
load_dotenv()


True

In [ ]:
# Bedrock呼び出し用のAPIクライアントを作成
client = boto3.client("bedrock-runtime")

input = "2025年7月の祝日はいつ？"
llm = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [4]:
def get_japanese_holidays(year: int) -> dict:
    """指定された年の日本の祝日を取得する"""
    url = f"https://holidays-jp.github.io/api/v1/{year}/date.json"
    with urllib.request.urlopen(url) as response:
        data = response.read()
        holidays = json.loads(data)
    return holidays

get_japanese_holidays(2025)

{'2025-01-01': '元日',
 '2025-01-13': '成人の日',
 '2025-02-11': '建国記念の日',
 '2025-02-23': '天皇誕生日',
 '2025-02-24': '天皇誕生日 振替休日',
 '2025-03-20': '春分の日',
 '2025-04-29': '昭和の日',
 '2025-05-03': '憲法記念日',
 '2025-05-04': 'みどりの日',
 '2025-05-05': 'こどもの日',
 '2025-05-06': 'みどりの日 振替休日',
 '2025-07-21': '海の日',
 '2025-08-11': '山の日',
 '2025-09-15': '敬老の日',
 '2025-09-23': '秋分の日',
 '2025-10-13': 'スポーツの日',
 '2025-11-03': '文化の日',
 '2025-11-23': '勤労感謝の日',
 '2025-11-24': '勤労感謝の日 振替休日'}

In [5]:
# 関数をLLMのツールとして定義
tools = [{
    "toolSpec": {
        "name": "get_japanese_holidays",
        "description": "指定された年の日本の祝日一覧を取得します",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "year": {
                        "type": "integer",
                        "description": "祝日を取得したい年（例: 2024）"
                    }
                },
                "required": ["year"]
            }
        }
    }
}]


In [6]:

# ========================================
# 1回目の推論
# ========================================
print("【推論1回目】")
print("ユーザーの入力： ", input)


【推論1回目】
ユーザーの入力：  2025年7月の祝日はいつ？


In [9]:

response = client.converse(
    modelId=llm,
    messages=[{
        "role": "user",
        "content": [{"text": input}]
    }],
    toolConfig={"tools": tools}
)

print(response)


{
    'ResponseMetadata': {
        'RequestId': '98b11fd3-2faa-42f5-bdf8-c5a2911cac9c',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'date': 'Fri, 14 Nov 2025 21:38:08 GMT',
            'content-type': 'application/json',
            'content-length': '534',
            'connection': 'keep-alive',
            'x-amzn-requestid': '98b11fd3-2faa-42f5-bdf8-c5a2911cac9c'
        },
        'RetryAttempts': 0
    },
    'output': {
        'message': {
            'role': 'assistant',
            'content': [
                {'text': '2025年7月の祝日を確認するために、日本の祝日情報を取得します。'},
                {
                    'toolUse': {
                        'toolUseId': 'tooluse_79IcJsptSG2GSd6xuqIxuQ',
                        'name': 'get_japanese_holidays',
                        'input': {'year': 2025},
                        'type': 'tool_use'
                    }
                }
            ]
        }
    },
    'stopReason': 'tool_use',
    'usage': {
        'inputTokens': 535,
        'outputTokens': 86,
        'totalTokens': 621,
        'cacheReadInputTokens': 0,
        'cacheWriteInputTokens': 0
    },
    'metrics': {'latencyMs': 1548}
}

In [10]:

# 初回のレスポンスを処理
message = response["output"]["message"]
print("LLMの回答： ", message["content"][0]["text"])


LLMの回答：  2025年7月の祝日を確認するために、日本の祝日情報を取得します。

In [16]:

# Tool Useの要否をチェック
tool_use = None
for content_item in message["content"]:
    if "toolUse" in content_item:
        tool_use = content_item["toolUse"]
        print("ツール要求： ", tool_use)
        print()
        break


ツール要求： 
{
    'toolUseId': 'tooluse_79IcJsptSG2GSd6xuqIxuQ',
    'name': 'get_japanese_holidays',
    'input': {'year': 2025},
    'type': 'tool_use'
}

In [19]:

# ========================================
# 2回目の推論
# ========================================

if tool_use:
    # 実際のツール実行：APIを呼び出して祝日を取得
    year = tool_use['input']['year']
    holidays = get_japanese_holidays(year)
    tool_result = {
        "year": year,
        "holidays": holidays,
        "count": len(holidays)
    }
    print("【アプリから直接、ツール実行して結果を取得】")
    print(tool_result)
    print()

    # 2回目の推論用に、1回目の会話履歴＋ツール実行結果を入力
    messages = [
        {
            "role": "user",
            "content": [{"text": input}] # 1回目の質問
        },
        {
            "role": "assistant",
            "content": message["content"] # 1回目の回答
        },
        {
            "role": "user",
            "content": [{
                "toolResult": {
                    "toolUseId": tool_use["toolUseId"],
                    "content": [{
                        "json": tool_result # ツール実行結果
                    }]
                }
            }]
        }
    ]
    print("messages: ", messages)

    # 2回目の回答
    final_response = client.converse(
        modelId=llm,
        messages=messages,
        toolConfig={"tools": tools}
    )
    output = final_response["output"]["message"]["content"][0]["text"]
    
    print("【推論2回目】")
    print("ユーザーの入力： （ツール実行結果）")
    print("LLMの回答： ", output)


【アプリから直接、ツール実行して結果を取得】

{
    'year': 2025,
    'holidays': {
        '2025-01-01': '元日',
        '2025-01-13': '成人の日',
        '2025-02-11': '建国記念の日',
        '2025-02-23': '天皇誕生日',
        '2025-02-24': '天皇誕生日 振替休日',
        '2025-03-20': '春分の日',
        '2025-04-29': '昭和の日',
        '2025-05-03': '憲法記念日',
        '2025-05-04': 'みどりの日',
        '2025-05-05': 'こどもの日',
        '2025-05-06': 'みどりの日 振替休日',
        '2025-07-21': '海の日',
        '2025-08-11': '山の日',
        '2025-09-15': '敬老の日',
        '2025-09-23': '秋分の日',
        '2025-10-13': 'スポーツの日',
        '2025-11-03': '文化の日',
        '2025-11-23': '勤労感謝の日',
        '2025-11-24': '勤労感謝の日 振替休日'
    },
    'count': 19
}

messages: 
[
    {'role': 'user', 'content': [{'text': '2025年7月の祝日はいつ？'}]},
    {
        'role': 'assistant',
        'content': [
            {'text': '2025年7月の祝日を確認するために、日本の祝日情報を取得します。'},
            {
                'toolUse': {
                    'toolUseId': 'tooluse_79IcJsptSG2GSd6xuqIxuQ',
                    'name': 'get_japanese_holidays',
                    'input': {'year': 2025},
                    'type': 'tool_use'
                }
            }
        ]
    },
    {
        'role': 'user',
        'content': [
            {
                'toolResult': {
                    'toolUseId': 'tooluse_79IcJsptSG2GSd6xuqIxuQ',
                    'content': [
                        {
                            'json': {
                                'year': 2025,
                                'holidays': {
                                    '2025-01-01': '元日',
                                    '2025-01-13': '成人の日',
                                    '2025-02-11': '建国記念の日',
                                    '2025-02-23': '天皇誕生日',
                                    '2025-02-24': '天皇誕生日 振替休日',
                                    '2025-03-20': '春分の日',
                                    '2025-04-29': '昭和の日',
                                    '2025-05-03': '憲法記念日',
                                    '2025-05-04': 'みどりの日',
                                    '2025-05-05': 'こどもの日',
                                    '2025-05-06': 'みどりの日 振替休日',
                                    '2025-07-21': '海の日',
                                    '2025-08-11': '山の日',
                                    '2025-09-15': '敬老の日',
                                    '2025-09-23': '秋分の日',
                                    '2025-10-13': 'スポーツの日',
                                    '2025-11-03': '文化の日',
                                    '2025-11-23': '勤労感謝の日',
                                    '2025-11-24': '勤労感謝の日 振替休日'
                                },
                                'count': 19
                            }
                        }
                    ]
                }
            }
        ]
    }
]

【推論2回目】

ユーザーの入力： （ツール実行結果）

LLMの回答：  2025年7月の祝日は以下の1日です：

- 7月21日（月）：海の日

海の日は日本の祝日で、海の恩恵に感謝するとともに、海洋国日本の繁栄を願う日とされています。2025年は7月21日の月曜日に
設定されています。